In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from utils import *
from model import SiameseNetwork
from loss import ContrastiveLoss
from dataset import CUBContrast

In [2]:
path = '/home/harsh/Downloads/CUB_200_2011/CUB_200_2011/'

## Initialize parameters
bs = 24
lr = 1e-3
threshold = 0.5
margin = 2
epochs = 100


In [3]:
## Initialize network
model = SiameseNetwork()
model = model.cuda()

## Initialize optimizer
optim = torch.optim.Adam(model.parameters(),lr=lr)

## Initialize scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optim,8)

## Initialize loss
criterion = ContrastiveLoss(margin)

In [4]:
## Initialize datasets and dataloaders
train_transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize(size=(224,224)),  # Let smaller edge match
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                             std=(0.229, 0.224, 0.225))
        ])
valid_transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize(size=(224,224)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                             std=(0.229, 0.224, 0.225))
        ])

test_transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize(size=(224,224)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                             std=(0.229, 0.224, 0.225))
        ])

train_ds = CUBContrast(ImageFolder(root = path + 'train',transform=train_transforms))
valid_ds = CUBContrast(ImageFolder(root = path + 'valid',transform=valid_transforms))
test_ds = CUBContrast(ImageFolder(root = path + 'test',transform=test_transforms))

train_dl = DataLoader(train_ds,batch_size=bs)
valid_dl = DataLoader(valid_ds,batch_size=bs)
test_dl = DataLoader(test_ds,batch_size=bs)

In [5]:
train_loss = []
valid_loss = []
for epoch in range(epochs):
    train_epoch_loss = 0
    model.train()    
    for i,(input1,input2,target) in enumerate(train_dl):
        optim.zero_grad()
        output1 = model(input1.cuda())
        output2 = model(input2.cuda())
        loss = criterion(output1,output2,target.cuda())
#         print(loss.item())
        train_epoch_loss += loss.item()
        train_loss.append(loss.item())
        loss.backward()
        optim.step()
#         scheduler.step()
        
    train_epoch_loss /= len(train_ds)
    valid_epoch_loss = 0
    val_accuracy = 0
#     model.eval()
#     for i,(input1,input2,target) in enumerate(valid_dl):
#         optim.zero_grad()
#         output1 = model(input1.cuda())
#         output2 = model(input2.cuda())
#         loss = criterion(output1,output2,target.cuda())
# #         print(loss.item())
#         valid_epoch_loss += loss.item()
#         valid_loss.append(loss.item())
#         val_accuracy += evaluate_pair(output1,output2,target.cuda(),0.5)
        
#     valid_epoch_loss /= len(valid_ds)
    
    print("Epoch [{}/{}] ----> Training loss :{} \tValidation loss :{} \n Validation Accuracy: {}".format(epoch+1,epochs,train_epoch_loss,valid_epoch_loss,val_accuracy/len(valid_ds)))

Epoch [1/100] ----> Training loss :0.023231718175077954 	Validation loss :0 
 Validation Accuracy: 0.0


KeyboardInterrupt: 

In [ ]:
from torchsummary import summary

In [ ]:
summary(model,(3,224,224),batch_size = 64)

In [ ]:
# for i in range(len(train_ds)):
#     print(i,train_ds[i][0].size(),train_ds[i][1].size(),)